In [11]:
import stim

def parse_detectors(circuit: stim.Circuit):
    """
    返回列表，每项格式：
    {
        'detector_id': 0..N-1,          # 电路里第几个 DETECTOR 指令
        'targets': [rec(-1), rec(-5)],  # 它监听的 measurement record
        'coords': (x,y,t),              # DETECTOR 后的坐标元组
        'round': t,                     # 这里直接用时间坐标当 round
        'measure_qubits': [],           # 下面会把 rec→实际测量 qubit 填进来
    }
    """
    det_list = []
    rec2qubit = []          # rec_id → 测量的是哪一比特
    rec_offset = 0          # 当前已加入的测量次数

    for inst in circuit:
        if inst.name == 'DETECTOR':
            det_list.append({
                'detector_id': len(det_list),
                'targets': inst.targets_copy(),
                'coords': inst.gate_args_copy(),
                'round': None,
                'measure_qubits': []
            })
        elif inst.name in ('M', 'MX', 'MY', 'MZ', 'MR', 'MRX', 'MRY', 'MRZ'):
            # 把这条指令里所有被测 qubit 登记到 rec2qubit
            for t in inst.targets_copy():
                if t.is_qubit_target:
                    rec2qubit.append(t.value)
            rec_offset += len([t for t in inst.targets_copy() if t.is_qubit_target])
        # 其它指令不管
    print(len(rec2qubit))

    # 第二遍：把 targets 里的 rec→qubit
    for det in det_list:
        for t in det['targets']:
            # stim 里 rec(-k) 用 target_rec(k-1) 表示
            if t.is_measurement_record_target:
                rec_id = t.value #- 1          # 0-based
                print('rec_id =', rec_id, 't.value =', t.value)
                det['measure_qubits'].append(rec2qubit[rec_id])
        # 坐标里最后一个分量当 round（常见惯例）
        det['round'] = int(det['coords'][-1]) if det['coords'] else 0

    return det_list

In [1]:
import numpy as np

def sample_to_4d(circuit, id2lrd, num_shots):
    sampler = circuit.compile_detector_sampler()
    # shape = (shots, num_detectors)
    flat = sampler.sample(num_shots, append_observables=False).astype(bool)

    # 先找出 logical 数、round 数、每轮最大 detections
    all_lrd = id2lrd.values()
    logicals = max(l for l, _, _ in all_lrd) + 1
    rounds   = max(r for _, r, _ in all_lrd) + 1
    max_d    = max(d for _, _, d in all_lrd) + 1

    # 建 4-D 空壳
    out = np.zeros((num_shots, logicals, rounds, max_d), dtype=bool)

    for flat_id, (l, r, d) in id2lrd.items():
        out[:, l, r, d] = flat[:, flat_id]

    return out

In [2]:
import collections

def build_lookup(dets, logical_coord_axis=0):
    """
    返回 dict: (logical, round, idx_within_round) -> flat_detector_id
    以及反向表
    """
    key2id = {}
    for det in dets:
        x = int(det['coords'][logical_coord_axis])
        r = det['round']
        key2id.setdefault((x, r), []).append(det['detector_id'])

    # 让 within-round 序号按 detector_id 升序排，保证稳定
    for k in key2id:
        key2id[k].sort()

    # 构造最终映射 (l,r,d) -> flat_id
    lrd2id = {}
    for (l, r), ids in key2id.items():
        for d, flat_id in enumerate(ids):
            lrd2id[(l, r, d)] = flat_id

    # 也返回反表，方便采样后重排
    id2lrd = {flat_id: (l, r, d) for (l, r, d), flat_id in lrd2id.items()}
    return lrd2id, id2lrd

In [12]:
circuit = stim.Circuit.generated("surface_code:rotated_memory_x",
                                 distance=5, rounds=5,
                                 after_clifford_depolarization=1e-3)
dets = parse_detectors(circuit)
_, id2lrd = build_lookup(dets, logical_coord_axis=0)
data4d = sample_to_4d(circuit, id2lrd, num_shots=100)

print("shape =", data4d.shape)          # (100, 5, 5, 10)  举例
print("logical 0, round 3 的 10 个 detector 触发情况")
print(data4d[0, 0, 3])

49
rec_id = -24 t.value = -24
rec_id = -16 t.value = -16
rec_id = -6 t.value = -6
rec_id = -21 t.value = -21
rec_id = -11 t.value = -11
rec_id = -2 t.value = -2
rec_id = -23 t.value = -23
rec_id = -14 t.value = -14
rec_id = -4 t.value = -4
rec_id = -19 t.value = -19
rec_id = -9 t.value = -9
rec_id = -1 t.value = -1
rec_id = -24 t.value = -24
rec_id = -25 t.value = -25
rec_id = -49 t.value = -49
rec_id = -14 t.value = -14
rec_id = -15 t.value = -15
rec_id = -19 t.value = -19
rec_id = -20 t.value = -20
rec_id = -41 t.value = -41
rec_id = -4 t.value = -4
rec_id = -5 t.value = -5
rec_id = -9 t.value = -9
rec_id = -10 t.value = -10
rec_id = -31 t.value = -31
rec_id = -18 t.value = -18
rec_id = -19 t.value = -19
rec_id = -23 t.value = -23
rec_id = -24 t.value = -24
rec_id = -46 t.value = -46
rec_id = -8 t.value = -8
rec_id = -9 t.value = -9
rec_id = -13 t.value = -13
rec_id = -14 t.value = -14
rec_id = -36 t.value = -36
rec_id = -3 t.value = -3
rec_id = -4 t.value = -4
rec_id = -27 t.value =

IndexError: index 3 is out of bounds for axis 2 with size 2

In [13]:
data4d.shape

(100, 9, 2, 3)

In [15]:
dets

[{'detector_id': 0,
  'targets': [stim.target_rec(-24)],
  'coords': [2.0, 0.0, 0.0],
  'round': 0,
  'measure_qubits': [3]},
 {'detector_id': 1,
  'targets': [stim.target_rec(-16)],
  'coords': [2.0, 4.0, 0.0],
  'round': 0,
  'measure_qubits': [20]},
 {'detector_id': 2,
  'targets': [stim.target_rec(-6)],
  'coords': [2.0, 8.0, 0.0],
  'round': 0,
  'measure_qubits': [42]},
 {'detector_id': 3,
  'targets': [stim.target_rec(-21)],
  'coords': [4.0, 2.0, 0.0],
  'round': 0,
  'measure_qubits': [9]},
 {'detector_id': 4,
  'targets': [stim.target_rec(-11)],
  'coords': [4.0, 6.0, 0.0],
  'round': 0,
  'measure_qubits': [31]},
 {'detector_id': 5,
  'targets': [stim.target_rec(-2)],
  'coords': [4.0, 10.0, 0.0],
  'round': 0,
  'measure_qubits': [51]},
 {'detector_id': 6,
  'targets': [stim.target_rec(-23)],
  'coords': [6.0, 0.0, 0.0],
  'round': 0,
  'measure_qubits': [5]},
 {'detector_id': 7,
  'targets': [stim.target_rec(-14)],
  'coords': [6.0, 4.0, 0.0],
  'round': 0,
  'measure_qubi